<p>
50.007 MACHINE LEARNING<br />
2024 SUMMER<br />
<b>COGITO</b><br />
</p>

# Setup

In [1]:
from base import *
from grid_search import cv_grid_search
from random_search import cv_random_search
from simulated_annealing import SimulatedAnnealing, cv_simulated_annealing

logging.basicConfig(
	filename="main.log",
	level=logging.INFO,
	format="%(asctime)s - %(message)s",
	datefmt="%Y-%m-%d %H:%M:%S"
)

# Task 1

In [ ]:
def σ(z): return 1 / (1 + np.exp(-z))
# def ce_loss(y, ŷ): return (-1/(y.shape[0])) * np.sum(y * np.log(ŷ) + (1 - y) * np.log(1 - ŷ))

# Return dw and db, for some X, y, ŷ, w, R, and λ.
def gradients_logreg(X, y, ŷ, w, R=None, λ=0):
	m, _ = X.shape
	dw = 1/m * np.dot(X.T, (ŷ - y))
	db = 1/m * np.sum(ŷ - y)
	if R == "L2":
		dw += λ * w / m
	elif R == "L1":
		dw += λ * np.sign(w) / m
	return dw, db

# Return (w, b) from gradient descent on X_train and y_train, for some τ, α, G, β, R, and λ.
def train_logreg(X_train, y_train, τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	m, n = X_train.shape
	w, b = np.zeros((n, 1)), 0
	for epoch in range(τ):
		if G == "full-batch":
			X_batch, y_batch = X_train, y_train
			ŷ = σ(np.dot(X_batch, w) + b)
			dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
			w, b = w - α*dw, b - α*db
		elif G == "mini-batch":
			for i in range(0, m, β):
				X_batch, y_batch = X_train[i:i+β], y_train[i:i+β]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
		elif G == "stochastic":
			for i in range(m):
				X_batch, y_batch = X_train[i:i+1], y_train[i:i+1]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
	return w, b

# Return array of predictions, where each prediction is 1 if corresponding ŷ entry > 0.5, and 0 otherwise.
def predict_logreg(wb_tuple, X):
	w, b = wb_tuple
	ŷ = σ(np.dot(X, w) + b)
	return np.array([1 if p > 0.5 else 0 for p in ŷ])

# Train model, make predictions, and save predictions to CSV file.
def generate_predictions_logreg(τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	start_time = time.time()
	w, b = train_logreg(np.array(X_train), np.array(y_train), τ, α, G, β, R, λ)
	predictions = predict_logreg((w, b), np.array(X_test))
	os.makedirs("./predictions/logreg/", exist_ok=True)
	file_name = os.path.join("./predictions/logreg/", f"τ={τ},α={α},G={G},β={β},R={R},λ={λ}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

In [ ]:
# generate_predictions_logreg(τ=1, α=0.08, G="mini-batch", β=128, R=None, λ=0)
# generate_predictions_logreg(τ=1000, α=0.08, G="stochastic", R=L2, λ=0.5)

## Task 2

In [ ]:
def apply_pca(x):
	scaler = StandardScaler()
	X_train_scaled = scaler.fit_transform(X_train)
	X_test_scaled = scaler.transform(X_test)
	if 0 <= x <= 1:
		# x is variance threshold.
		pca = PCA(n_components=None)
		pca.fit(X_train_scaled)
		c = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= x) + 1
		v = x
	else:
		# x is number of components.
		pca = PCA(n_components=x)
		pca.fit(X_train_scaled)
		c = x
		v = sum(pca.explained_variance_ratio_)
	# Transform train and test datasets.
	X_train_pca = pca.transform(X_train_scaled)
	X_test_pca = pca.transform(X_test_scaled)
	return X_train_pca, X_test_pca, c, v

def train_SKLknn(X_train, y_train, k=5, W="uniform", p=2, m="minkowski"):
	model = KNeighborsClassifier(
		n_neighbors=k,
		weights=W,
		p=p,
		metric=m,
		n_jobs=-1
	)
	model.fit(X_train, y_train)
	return model

def predict_SKLknn(model, X): return model.predict(X)

# Train model, make model predictions, and save model predictions to CSV file.
def generate_predictions_pcaknn(x):
	start_time = time.time()
	X_train_pca, X_test_pca, c, v = apply_pca(x)
	model = train_SKLknn(X_train_pca, y_train, k=2)
	predictions = predict_SKLknn(model, X_test_pca)
	os.makedirs("./predictions/pcaknn/", exist_ok=True)
	file_name = os.path.join("./predictions/pcaknn/", f"pcaknn(ρ={c},ν={v:.2f}).csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

In [ ]:
# generate_predictions_pcaknn(100)
# generate_predictions_pcaknn(500)
# generate_predictions_pcaknn(1000)
# generate_predictions_pcaknn(2000)
# generate_predictions_pcaknn(5000)

## Task 3

## SKLknn

In [ ]:
# Default:
# generate_predictions(
# 	"SKLknn",
# 	n_neighbors=5,
# 	weights="uniform",
# 	p=2,
# 	metric="minkowski"
# )

## SKLrf

In [ ]:
# Default:
# generate_predictions(
# 	"SKLrf",
# 	n_estimators=100,
# 	criterion="gini",
# 	max_depth=None,
# 	min_samples_split=2,
# 	min_samples_leaf=1,
# 	max_leaf_nodes=None,
# 	random_state=None
# )

## SKLsvm

In [ ]:
# Default:
# generate_predictions(
# 	"SKLsvm",
# 	C=1.0,
# 	kernel="rbf",
# 	degree=3,
# 	gamma="scale",
# 	max_iter=-1,
# 	random_state=None
# )

## SKLgb

In [ ]:
# Default:
# generate_predictions(
# 	"SKLgb",
# 	learning_rate=0.1,
# 	n_estimators=100,
# 	subsample=1.0,
# 	min_samples_split=2,
# 	min_samples_leaf=1,
# 	max_depth=3,
# 	random_state=None,
# 	max_leaf_nodes=None
# )

## SKLhgb

In [ ]:
# Default:
# generate_predictions(
# 	"SKLhgb",
# 	learning_rate=0.1,
# 	max_iter=100,
# 	max_leaf_nodes=31,
# 	max_depth=None,
# 	min_samples_leaf=20,
# 	l2_regularization=0.0,
# 	random_state=None
# )

generate_predictions(
	"SKLhgb",
	learning_rate=0.08,
	max_iter=500,
	max_leaf_nodes=75,
	max_depth=None,
	min_samples_leaf=15,
	l2_regularization=0.5
)

# cv_grid_search("SKLhgb", grid={
# 	"learning_rate": [0.08],
# 	"max_depth": [80]
# 	"max_iter": [500],
# 	"max_leaf_nodes": [70, 75, 80],
# 	"min_samples_leaf": [15, 18, 20]
# }, k=5)

## XGBgb

In [ ]:
# Default:
# generate_predictions(
# 	"XGBgb",
# 	n_estimators=500,
# 	max_depth=8,
# 	learning_rate=0.1,
# 	objective="binary:logistic",
# 	booster="gbtree",
# 	reg_alpha=0.1,
# 	reg_lambda=2,
# 	random_state=None
# )

generate_predictions(
	"XGBgb",
	n_estimators=800,
	learning_rate=0.08,
	max_depth=8,
	reg_alpha=0.1,
	reg_lambda=1
)

# cv_grid_search("XGBgb", grid={
# 	"n_estimators": [600],
# 	"learning_rate": [0.1],
# 	"reg_alpha": [0.1],
# 	"reg_lambda": [1, 2],
# 	"subsample": [0.73],
# 	"min_child_weight": [2],
# 	"gamma": [2.77],
# 	"colsample_bytree": [0.6]
# }, k=5)

## CBgb

In [4]:
# Default:
# generate_predictions(
# 	"CBgb",
# 	learning_rate=0.03,
# 	subsample=0.8,
# 	max_depth=6,
# 	n_estimators=1000,
# 	reg_lambda=3.0,
# 	random_state=None
# )

cv_grid_search("CBgb", grid={
	"n_estimators": [680, 700, 720],
	"learning_rate": [0.05],
	"reg_lambda": [2.0, 2.4, 2.8]
}, k=5)

# cv_random_search("CBgb", grid={
# 	"learning_rate": uniform(0.001, 0.1),
# 	"max_depth": randint(1, 2),
# 	"n_estimators": randint(1, 2),
# }, k=2, n_iter=1)

# cv_simulated_annealing(model_name="CBgb", grid={
# 	"learning_rate": (0.06, 0.12),
# 	"max_depth": [12],
# 	"reg_lambda": (0.1, 1)
# })

Fitting 5 folds for each of 9 candidates, totalling 45 fits
0:	learn: 0.6870685	total: 31.7ms	remaining: 21.5s
1:	learn: 0.6818565	total: 49.6ms	remaining: 16.8s
2:	learn: 0.6774244	total: 65.3ms	remaining: 14.7s
3:	learn: 0.6737035	total: 81.9ms	remaining: 13.8s
4:	learn: 0.6696910	total: 97.5ms	remaining: 13.2s
5:	learn: 0.6658551	total: 113ms	remaining: 12.7s
6:	learn: 0.6625288	total: 129ms	remaining: 12.4s
7:	learn: 0.6592701	total: 145ms	remaining: 12.2s
8:	learn: 0.6567658	total: 161ms	remaining: 12s
9:	learn: 0.6541407	total: 177ms	remaining: 11.8s
10:	learn: 0.6517118	total: 193ms	remaining: 11.7s
11:	learn: 0.6495894	total: 209ms	remaining: 11.6s
12:	learn: 0.6476689	total: 224ms	remaining: 11.5s
13:	learn: 0.6456646	total: 241ms	remaining: 11.5s
14:	learn: 0.6438950	total: 257ms	remaining: 11.4s
15:	learn: 0.6419272	total: 275ms	remaining: 11.4s
16:	learn: 0.6405102	total: 292ms	remaining: 11.4s
17:	learn: 0.6390801	total: 307ms	remaining: 11.3s
18:	learn: 0.6371051	total: 3

(<catboost.core.CatBoostClassifier at 0x128d9da90>,
 {'learning_rate': 0.05, 'n_estimators': 720, 'reg_lambda': 2.0})

## SKLstack

In [ ]:
# Join after
generate_predictions(
	"SKLstack",
	verbose=2
)

In [ ]:
cv_grid_search("SKLstack", grid={
	"penalty": ["l2"],
	"C": [1]
}, k=5)